In [32]:
import math
import torch
def polar_to_cartesian(theta, phi):
    x = math.sin(math.radians(phi)) * math.cos(math.radians(theta))
    y = math.sin(math.radians(phi)) * math.sin(math.radians(theta))
    z = math.cos(math.radians(phi))
    return [x, y, z]

In [33]:
# 示例输入
theta = 45
phi = 90

# 转换为归一化的笛卡尔坐标
[x, y, z] = polar_to_cartesian(theta, phi)
print("笛卡尔坐标:", x, y, z)
print(x*x+y*y+z*z)

笛卡尔坐标: 0.7071067811865476 0.7071067811865475 6.123233995736766e-17
1.0


In [34]:
in_em = [45,90,2] #\theta, \phi, ka
incident_wave_vec = polar_to_cartesian(in_em[0],in_em[1])
print(incident_wave_vec)
print(torch.tensor(incident_wave_vec).shape)

[0.7071067811865476, 0.7071067811865475, 6.123233995736766e-17]
torch.Size([3])


In [35]:
# 指定新矩阵的大小
new_size = [1, 33564 ,1]

# 使用 torch.repeat() 函数复制矢量
out = torch.tensor(incident_wave_vec).repeat(*new_size)

print(out.size())  # 打印新矩阵的大小
print(out.shape)
print(out)

torch.tensor(incident_wave_vec).repeat(new_size)

torch.Size([1, 33564, 3])
torch.Size([1, 33564, 3])
tensor([[[7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         ...,
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17]]])


tensor([[[7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         ...,
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17]]])

In [36]:
incident_freq_mtx = torch.tensor(in_em[2]).repeat(torch.Size([1, 33564, 1]))
print(incident_freq_mtx)
print(incident_freq_mtx.shape)

tensor([[[2],
         [2],
         [2],
         ...,
         [2],
         [2],
         [2]]])
torch.Size([1, 33564, 1])


In [37]:

from math import ceil, pi, sqrt, degrees
import math

import torch
from torch import nn, Tensor, einsum
from torch.nn import Module, ModuleList
import torch.nn.functional as F
from torch.utils.checkpoint import checkpoint
from torch.cuda.amp import autocast

from torchtyping import TensorType

from pytorch_custom_utils import save_load

from beartype import beartype
from beartype.typing import Union, Tuple, Callable, Optional, List, Dict, Any

from einops import rearrange, repeat, reduce, pack, unpack
from einops.layers.torch import Rearrange

from einx import get_at

from x_transformers import Decoder
from x_transformers.attend import Attend
from x_transformers.x_transformers import RMSNorm, FeedForward, LayerIntermediates

from x_transformers.autoregressive_wrapper import (
    eval_decorator,
    top_k,
    top_p,
)

from local_attention import LocalMHA

from vector_quantize_pytorch import (
    ResidualVQ,
    ResidualLFQ
)

from net.data import derive_face_edges_from_faces, myderive_face_edges_from_faces
from net.version import __version__

from taylor_series_linear_attention import TaylorSeriesLinearAttn

from classifier_free_guidance_pytorch import (
    classifier_free_guidance,
    TextEmbeddingReturner
)

from torch_geometric.nn.conv import SAGEConv

from gateloop_transformer import SimpleGateLoopLayer

from tqdm import tqdm

# helper functions

def exists(v):
    return v is not None

def default(v, d):
    return v if exists(v) else d

def first(it):
    return it[0]

def divisible_by(num, den):
    return (num % den) == 0

def is_odd(n):
    return not divisible_by(n, 2)

def is_empty(l):
    return len(l) == 0

def is_tensor_empty(t: Tensor):
    return t.numel() == 0

def set_module_requires_grad_(
    module: Module,
    requires_grad: bool
):
    for param in module.parameters():
        param.requires_grad = requires_grad

def l1norm(t):
    return F.normalize(t, dim = -1, p = 1)

def l2norm(t):
    return F.normalize(t, dim = -1, p = 2)

def safe_cat(tensors, dim):
    tensors = [*filter(exists, tensors)]

    if len(tensors) == 0:
        return None
    elif len(tensors) == 1:
        return first(tensors)

    return torch.cat(tensors, dim = dim)

def pad_at_dim(t, padding, dim = -1, value = 0):
    ndim = t.ndim
    right_dims = (ndim - dim - 1) if dim >= 0 else (-dim - 1)
    zeros = (0, 0) * right_dims
    return F.pad(t, (*zeros, *padding), value = value)

def pad_to_length(t, length, dim = -1, value = 0, right = True):
    curr_length = t.shape[dim]
    remainder = length - curr_length

    if remainder <= 0:
        return t

    padding = (0, remainder) if right else (remainder, 0)
    return pad_at_dim(t, padding, dim = dim, value = value)



In [40]:
def vector_anglejxt(x, y, eps = 1e-5): #给矢量
    normdot = -torch.einsum('...d,...d->...', l2norm(x), l2norm(y)) 
    radians = normdot.clip(-1 + eps, 1 - eps).arccos()
    angle = torch.tensor([[[degrees(row.item())] for row in matrix] for matrix in radians])
    return radians, angle

In [41]:
normals = torch.tensor([[[-0.6752, -0.7332, -0.0809],
         [-0.6926, -0.7013, -0.1688],
         [-0.6796, -0.7101, -0.1840],
         [ 0.9110, -0.0677,  0.4068],
         [ 0.9522,  0.3035,  0.0348],
         [ 0.9336,  0.3351,  0.1272]]])

incident_angle_mtx = torch.tensor([[[7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17],
         [7.0711e-01, 7.0711e-01, 6.1232e-17]]])

incident_mesh_anglehudu, incident_mesh_anglejiaodu = vector_anglejxt(normals, incident_angle_mtx)
print(incident_mesh_anglehudu)
print(incident_mesh_anglejiaodu)

tensor([[0.0908, 0.1697, 0.1863, 2.2097, 2.6636, 2.6839]])
tensor([[[  5.2039],
         [  9.7245],
         [ 10.6765],
         [126.6058],
         [152.6118],
         [153.7756]]])
